
NLP has a goal of deriving information out of our natural language (could be sequences text or speech)

Another common word for NLP problems is sequence to sequence problems(seq2seq)




In [ ]:
#check the GPU
!nvidia-smi

Thu Jul 13 15:13:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Get helper functions

In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-07-13 15:13:54--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-07-13 15:13:54 (114 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
from helper_functions import unzip_data, plot_loss_curves, compare_historys,walk_through_dir,create_tensorboard_callback

## Get a text dataset

The data we're going to use is kaggle's introduction to NLP dataset (text samples of tweet labels as disaster or not a disaster)

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

#extract the zip file
unzip_data('nlp_getting_started.zip')

--2023-07-13 15:13:59--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.79.128, 108.177.119.128, 108.177.126.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.79.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  1.46MB/s    in 0.4s    

2023-07-13 15:13:59 (1.46 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



##visualizing a text dataset

we can load dataset with python ...
we can use pandas

but pandas may need some space in your ram to store the data
so you can even try with tensorflow load method

In [ ]:
import pandas as pd

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df.head(5)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
#shuffle the training data
train_data_shuffled = train_df.sample(frac = 1, random_state = 42)
train_data_shuffled.head(5)

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [ ]:
train_data_shuffled['text']

2644    So you have a new weapon that can cause un-ima...
2227    The f$&amp;@ing things I do for #GISHWHES Just...
5448    DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...
132     Aftershock back to school kick off was great. ...
6845    in response to trauma Children of Addicts deve...
                              ...                        
5226    @Eganator2000 There aren't many Obliteration s...
5390    just had a panic attack bc I don't have enough...
860     Omron HEM-712C Automatic Blood Pressure Monito...
7603    Officials say a quarantine is in place at an A...
7270    I moved to England five years ago today. What ...
Name: text, Length: 7613, dtype: object

In [ ]:
#how many examples of each class ?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [ ]:
import random

random_index = random.randint(0,len(train_df)-5)# create random indexes not higher than the total number of samples
for row in train_data_shuffled[['text','target']][random_index:random_index+5].itertuples():
  _,text,target = row
  print(f'Target: {target}', 'real diaster ' if target > 0 else "not real diaster" )
  print(f'Text: \n{text}\n')
  print('---\n')


Target: 1 real diaster 
Text: 
Major accident causes life-threatening injuries closes highway: An 18-wheeler and an SUV collided and thenÛ_ http://t.co/ajTXUafOEM

---

Target: 0 not real diaster
Text: 
Add Familia to the arson squad.

---

Target: 1 real diaster 
Text: 
Emergency crews respond to chemical spill downtown beaumont #benews http://t.co/PME0HOJVYA

---

Target: 0 not real diaster
Text: 
Uribe just annihilated that baseball. #Mets

---

Target: 1 real diaster 
Text: 
Evacuation order lifted for town of Roosevelt: http://t.co/EDyfo6E2PU http://t.co/M5KxLPKFA1

---



###Split data into Training and validation sets

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#use train_test_split to split training data into training and validation sets
train_sentences, val_sentences , train_labels , val_labels = train_test_split(train_data_shuffled['text'].to_numpy(),
                                                                              train_data_shuffled['target'].to_numpy(),
                                                                              test_size = 0.1, #use 10% of training data for validation split
                                                                              random_state = 42)

In [ ]:
#check the lengths
len(train_sentences), len(val_sentences), len(train_labels) ,len(val_labels)

(6851, 762, 6851, 762)

## Text vectorization (tokenization)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

#use the default textvectorization parameters
text_vectorizer = TextVectorization(max_tokens = None, #how many words in the vocabulary (if you put 10k then it will find the words in 10k otherwise it will give <oov>)
                                    standardize = 'lower_and_strip_punctuation',
                                    split = 'whitespace',
                                    ngrams = None, #create group of words
                                    output_mode = 'int' , #how to map tokens to numbers
                                    output_sequence_length = None ,#how long do you want your sequences to be?
                                    pad_to_max_tokens = False )

In [ ]:
train_sentences[0].split()

['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet']

In [ ]:
#find the average number of words from the tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))   #add the words and divide them with total number of sentences


15

In [ ]:
#setup text vectorization variables
max_vocab_length = 10000 #max number of words to have in our vocabulary
max_length = 15 #max length our sequences will be (how many words from a tweet does a model see?)

text_vectorizer = TextVectorization(max_tokens =max_vocab_length,
                                    output_mode = 'int',
                                    output_sequence_length  = max_length)



In [ ]:
#fit the text vectorizer to the data
text_vectorizer.adapt(train_sentences)

In [ ]:
#create a sample sentence and tokenize it
sample_sentence = "There's flood in my street"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [ ]:
#choose a random sentence from the training data
random_sentence = random.choice(train_sentences)
print(f'Original sentence :\n {random_sentence} \
       \n\n Vectorized sentence: \n {text_vectorizer([random_sentence])}')

Original sentence :
 When I breathe it sounds like a windstorm. Haha cool        

 Vectorized sentence: 
 [[  45    8 4060   15  919   25    3  428  847  607    0    0    0    0
     0]]


In [ ]:
#Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() #gets all the unique words in the training data
top_5_words = words_in_vocab[:5] #gives the top 5 words
bottom_5_words = words_in_vocab[-5:]
print(f'Number of words in vocab : \n {len(words_in_vocab)}')
print(f'Top 5 words : \n {top_5_words} ,\n \n Bottom 5 words : \n {bottom_5_words}')

Number of words in vocab : 
 10000
Top 5 words : 
 ['', '[UNK]', 'the', 'a', 'in'] ,
 
 Bottom 5 words : 
 ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using an Embedding layer



In [ ]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim = max_vocab_length, #set input shape
                             output_dim = 128, #set up the number which are divisible by 8  (output shape)
                             input_length  = max_length)

embedding

In [ ]:
#getting random sentences
random_sentence = random.choice(train_sentences)
print(f'Original sentence :\n\n {random_sentence} \
      \n\n Embedded version: \n')

#embed the random sentence (turn it into dense vector of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original sentence :

 Now Trending in Nigeria: Police charge traditional ruler others with informantÛªs  murder http://t.co/93inFxzhX0       

 Embedded version: 



<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.03760818,  0.01032579, -0.01561196, ..., -0.00184443,
         -0.03526325, -0.00755435],
        [-0.02716541, -0.04269552,  0.03474666, ...,  0.01582427,
         -0.04412213, -0.0273567 ],
        [ 0.03572446, -0.01830507,  0.01615635, ..., -0.03775961,
          0.01064108,  0.03422748],
        ...,
        [-0.02648433,  0.01454226, -0.01168177, ...,  0.01277299,
         -0.01335979,  0.0273917 ],
        [-0.02491432, -0.02250677, -0.04887626, ...,  0.04942216,
          0.02730164,  0.01338449],
        [-0.02491432, -0.02250677, -0.04887626, ...,  0.04942216,
          0.02730164,  0.01338449]]], dtype=float32)>

In [ ]:
#check out a single embedding token
sample_embed[0][0] , sample_embed[0][0].shape , random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.03760818,  0.01032579, -0.01561196, -0.04751791, -0.01616591,
         0.04889164, -0.01771667,  0.00758543,  0.01853863,  0.01484754,
        -0.03634589,  0.04544797,  0.0030605 ,  0.03684013,  0.01313328,
        -0.01329547,  0.03011525,  0.0096076 , -0.03115573,  0.01707089,
        -0.00742792, -0.0087787 , -0.00860146,  0.03432448,  0.03735414,
        -0.04933184, -0.01428162, -0.02209692, -0.00551796,  0.02909419,
         0.0395101 , -0.01660346,  0.00570067,  0.03047539, -0.01870089,
        -0.00576442,  0.02866052, -0.04420488,  0.04374653, -0.03662366,
         0.0418837 ,  0.01680103,  0.04240383, -0.00405801, -0.00904647,
        -0.02027338, -0.02471737, -0.02844896, -0.00882189,  0.00633043,
        -0.01332565,  0.03375847,  0.00787073,  0.01762382, -0.03110803,
        -0.04839525,  0.04723105,  0.04156568, -0.02232518, -0.02789532,
        -0.02723485, -0.00943401, -0.04939649,  0.00400443, -0.00395523,
  

In [ ]:
## Modelling a text dataset

In [ ]:
### Model 0: Getting a baseline

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

In [ ]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

In [ ]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

In [ ]:
### Creating an evaluation function for our model experiments

In [ ]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [ ]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

## Model 1 : A simple dense model

In [ ]:
# Create tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create directory to save TensorBoard logs
SAVE_DIR = "model_logs"

In [ ]:
#now we got a tensor callback function let's create the model
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numerized numbers
x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding (try running the model without this layer and see what happens)
outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model

In [ ]:
# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
# Get a summary of the model
model_1.summary()

In [ ]:
# Fit the model
model_1_history = model_1.fit(train_sentences, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR, 
                                                                     experiment_name="simple_dense_model")])

In [ ]:
#evaluate the model
# Check the results
model_1.evaluate(val_sentences, val_labels)

In [ ]:
embedding.weights

In [ ]:
embed_weights = model_1.get_layer("embedding_1").get_weights()[0]
print(embed_weights.shape)

In [ ]:
# Make predictions (these come back in the form of probabilities)
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[:10] # only print out the first 10 prediction probabilities

In [ ]:
# Turn prediction probabilities into single-dimension tensor of floats
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs)) # squeeze removes single dimensions
model_1_preds[:20]

In [ ]:
# Calculate model_1 metrics
model_1_results = calculate_results(y_true=val_labels, 
                                    y_pred=model_1_preds)
model_1_results

## Model 2 : LSTM

In [ ]:
# Set random seed and create embedding layer (new embedding layer for each model)
tf.random.set_seed(42)
from tensorflow.keras import layers
model_2_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_2")


# Create LSTM model
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_2_embedding(x)
print(x.shape)
# x = layers.LSTM(64, return_sequences=True)(x) # return vector for each word in the Tweet (you can stack RNN cells as long as return_sequences=True)
x = layers.LSTM(64)(x) # return vector for whole sequence
print(x.shape)
# x = layers.Dense(64, activation="relu")(x) # optional dense layer on top of output of LSTM cell
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

In [ ]:
# Compile model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
# Fit model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, 
                                                                     "LSTM")])

In [ ]:
# Make predictions on the validation dataset
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs.shape, model_2_pred_probs[:10] # view the first 10

In [ ]:
# Round out predictions and reduce to 1-dimensional array
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

In [ ]:
# Calculate LSTM model results
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)
model_2_results

In [ ]:
# Calculate LSTM model results
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)
model_2_results

## Model 3: GRU

In [ ]:
# Set random seed and create embedding layer (new embedding layer for each model)
tf.random.set_seed(42)
from tensorflow.keras import layers
model_3_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_3")

In [ ]:
# Build an RNN using the GRU cell
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_3_embedding(x)
# x = layers.GRU(64, return_sequences=True) # stacking recurrent cells requires return_sequences=True
x = layers.GRU(64)(x) 
# x = layers.Dense(64, activation="relu")(x) # optional dense layer after GRU cell
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

In [ ]:
# Compile GRU model
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
# Get a summary of the GRU model
model_3.summary()

In [ ]:
# Fit model
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "GRU")])

In [ ]:
# Make predictions on the validation data
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs.shape, model_3_pred_probs[:10]

In [ ]:

# Convert prediction probabilities to prediction classes
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

In [ ]:
# Calcuate model_3 results
model_3_results = calculate_results(y_true=val_labels, 
                                    y_pred=model_3_preds)
model_3_results